In [1]:
from lxml import html  
import csv,os,json
import requests
import pandas as pd
from time import sleep
#import socket
from selenium import webdriver
#import socks

In [24]:
#socks.setdefaultproxy(proxy_type=socks.PROXY_TYPE_SOCKS5, addr="127.0.0.1", port=9050)
#socket.socket = socks.socksocket
#driver = webdriver.Firefox(executable_path=r'C:\Geckodriver\geckodriver.exe')
#driver = webdriver.Chrome(executable_path=r'C:\Geckodriver\chromedriver.exe')
#driver.set_page_load_timeout(100)
#print(requests.get("http://icanhazip.com").text)
#print(driver.get("http://icanhazip.com").text)
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
driver = webdriver.Firefox(executable_path=r'C:\Geckodriver\geckodriver.exe')
driver.set_page_load_timeout(100)

In [25]:
def AmzonParser(url):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
    driver.get(url)
    #print(driver)
    rpage = requests.get(url,headers=headers)
    page = driver.page_source
    while True:
        sleep(3)
        try:
            doc = html.fromstring(page)
            XPATH_NAME = '//h1[@id="title"]//text()'
            #XPATH_SALE_PRICE = '//span[contains(@id,"ourprice") or contains(@id,"saleprice")]/text()'
            XPATH_SALE_PRICE = '//span[contains(@class,"offer-price") and contains(@class,"a-size-medium")]//text()'
            #XPATH_ORIGINAL_PRICE = '//td[contains(text(),"List Price") or contains(text(),"M.R.P") or contains(text(),"Price")]/following-sibling::td/text()'
            XPATH_CATEGORY = '//a[@class="a-link-normal a-color-tertiary"]//text()'
            XPATH_AVAILABILITY = '//div[@id="availability"]//text()' 
            XPATH_AUTHOR = '//a[contains(@class,"contributorNameID")]//text()'
            XPATH_ISBN10 = '//li/b[contains(text(),"ISBN-10:")]/following-sibling::text()'
            XPATH_PAGES = '//li/b[contains(text(),"Paperback") or contains(text(),"Hardcover") or contains(text(),"Print Length")]/following-sibling::text()'
            XPATH_PUBLISHER = '//li/b[contains(text(),"Publisher")]/following-sibling::text()'
            XPATH_LANGUAGE = '//li/b[contains(text(),"Language")]/following-sibling::text()'
            XPATH_DIMENSIONS = '//li/b[contains(text(),"Dimensions")]/following-sibling::text()'
            XPATH_WEIGHT = '//li/b[contains(text(),"Weight")]/following-sibling::text()'
            XPATH_RATING = '//li/b[contains(text(),"Average Customer Review:")]/following-sibling::span/span/a/i/span/text()'
           
            RAW_NAME = doc.xpath(XPATH_NAME)
            RAW_SALE_PRICE = doc.xpath(XPATH_SALE_PRICE)
            RAW_CATEGORY = doc.xpath(XPATH_CATEGORY)
            ##RAW_ORIGINAL_PRICE = doc.xpath(XPATH_ORIGINAL_PRICE)
            RAw_AVAILABILITY = doc.xpath(XPATH_AVAILABILITY)
            RAW_AUTHOR = doc.xpath(XPATH_AUTHOR)
            RAW_ISBN10 = doc.xpath(XPATH_ISBN10)
            RAW_PAGES = doc.xpath(XPATH_PAGES)
            RAW_PUBLISHER = doc.xpath(XPATH_PUBLISHER) 
            RAW_LANGUAGE = doc.xpath(XPATH_LANGUAGE)
            RAW_DIMENSIONS = doc.xpath(XPATH_DIMENSIONS)
            RAW_WEIGHT = doc.xpath(XPATH_WEIGHT)
            RAW_RATING = doc.xpath(XPATH_RATING)
            
            print(RAW_RATING)
 
            NAME = ' '.join(''.join(RAW_NAME).split()) if RAW_NAME else None
            SALE_PRICE = ' '.join(''.join(RAW_SALE_PRICE).split()).strip() if RAW_SALE_PRICE else None
            CATEGORY = ' > '.join([i.strip() for i in RAW_CATEGORY]) if RAW_CATEGORY else None
            #ORIGINAL_PRICE = ''.join(RAW_ORIGINAL_PRICE).strip() if RAW_ORIGINAL_PRICE else None
            AVAILABILITY = ''.join(RAw_AVAILABILITY).strip() if RAw_AVAILABILITY else None
            AUTHOR = ''.join(RAW_AUTHOR).strip() if RAW_AUTHOR else None
            ISBN10 = ''.join(RAW_ISBN10).strip() if RAW_ISBN10 else None
            PAGES = ''.join(RAW_PAGES).strip() if RAW_PAGES else None
            PUBLISHER = ''.join(RAW_PUBLISHER).strip() if RAW_PUBLISHER else None
            LANGUAGE = ''.join(RAW_LANGUAGE).strip() if RAW_LANGUAGE else None
            DIMENSIONS = ''.join(RAW_DIMENSIONS).strip() if RAW_DIMENSIONS else None
            WEIGHT = ''.join(RAW_WEIGHT).strip().strip('()').strip() if RAW_WEIGHT else None
            RATING = ''.join(RAW_RATING).strip() if RAW_RATING else None
            #if not ORIGINAL_PRICE:
                #ORIGINAL_PRICE = SALE_PRICE
 
            if rpage.status_code!=200:
                raise ValueError('captha')
            data = {
                    'NAME':NAME,
                    'SALE_PRICE':SALE_PRICE,
                    'CATEGORY':CATEGORY,
                    #'ORIGINAL_PRICE':ORIGINAL_PRICE,
                    'AVAILABILITY':AVAILABILITY,
                    'URL':url,
                    'AUTHOR' : AUTHOR,
                    'ISBN10' : ISBN10,
                    'PAGES' : PAGES,
                    'PUBLISHER' : PUBLISHER,
                    'LANGUAGE' : LANGUAGE,
                    'DIMENSIONS' : DIMENSIONS,
                    'WEIGHT' : WEIGHT,
                    'RATING' : RATING,
                    }
 
            return data
        except Exception as e:
            print(e)
 

In [26]:
def ReadAsin(AsinList):
    # AsinList = csv.DictReader(open(os.path.join(os.path.dirname(__file__),"Asinfeed.csv")))
    extracted_data = []
    for i in AsinList:
        url = "http://www.amazon.com/gp/product/"+i
        print("Processing: "+url)
        extracted_data.append(AmzonParser(url))
        sleep(5)
    f=open('data.json','w')
    json.dump(extracted_data,f,indent=4)
 

In [27]:
formatted_asin = []
df = pd.read_csv("amazon_product_asins.csv")
for index, row in df.iterrows():
    if len(row[0]) < 8:
        asin = "000" + row[0]
        formatted_asin.append(asin)
    elif len(row[0]) < 9:
        asin = "00" + row[0]
        formatted_asin.append(asin)
    elif len(row[0]) < 10:
        asin = "0" + row[0]
        formatted_asin.append(asin)
    else:
        formatted_asin.append(row[0])

In [23]:
if __name__ == "__main__":
    ReadAsin(formatted_asin[1500:2000])

Processing: http://www.amazon.com/gp/product/1684151325
['5.0 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1632156741
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0812540115
['4.7 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0062468979
[]
Processing: http://www.amazon.com/gp/product/0399179887
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0307950689
['4.0 out of 5 stars']
Processing: http://www.amazon.com/gp/product/038572179X
['3.9 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1977061451
['4.7 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0778330958
['4.2 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0486278077
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0515142247
['4.4 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1538744406
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0399576606
['4.4 out of 5 s

['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0812976533
['3.9 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1250102685
[]
Processing: http://www.amazon.com/gp/product/184918352X
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0316363936
['4.8 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0147515874
['4.4 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1534303456
['4.0 out of 5 stars']
Processing: http://www.amazon.com/gp/product/178681143X
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0451232852
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1939366976
[]
Processing: http://www.amazon.com/gp/product/1250105560
['4.4 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0425260933
['4.1 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1471162842
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1501156705
['4.6 out of 

['4.7 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1981816828
['5.0 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1401268544
['4.0 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0451494628
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0544838262
['4.7 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0765368544
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1401277578
['5.0 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0765397536
['4.7 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0062259660
['4.4 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0425269051
['4.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0143039989
['3.9 out of 5 stars']
Processing: http://www.amazon.com/gp/product/147678938X
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1501134132
['4.5 out of 5 stars']
Processing: http://www.amazon

['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1401227996
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0899190359
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0425285529
['4.5 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0147517729
['4.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1546368590
['4.7 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0765378566
['4.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1473204305
['4.1 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1936781565
['4.8 out of 5 stars']
Processing: http://www.amazon.com/gp/product/0143131842
['4.3 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1455512869
['4.8 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1501166220
['4.6 out of 5 stars']
Processing: http://www.amazon.com/gp/product/1617734497
['4.3 out of 5 stars']
Processing: http://www.amazon